In [1]:
import pandas as pd 
import numpy as np

In [57]:
headlines = pd.read_csv('../data/processed/headlines_cl_sent_pol_rapi.csv')
headlines['year'] = pd.to_datetime(headlines['time']).dt.year

In [140]:
df = headlines[(headlines['year']>2010)& (headlines['country_of_pub']=='UK')]
list(df[df['headline'].str.contains(' wedding ')]['headline'].drop_duplicates())

l and her grandma buy matching dresses for a family wedding and it's so cute - Metro",
 'Benefit cheat Wendy Cooper who said she was single jailed after posting wedding photos on Facebook - Metro',
 "Grown man is his cousin's 'flower girl' at her wedding and completely bosses it - Metro",
 'Natalie Sideserf creates gruesome severed head wedding cake - Metro',
 'Ryan Leak organises dream wedding for Amanda Roman through Pinterest - Metro',
 "Prime minister David Cameron snapped taking a snooze at sister-in-law's wedding - Metro",
 'Woman told she was cancer free dies three months after wedding - Metro',
 'Bride developed vitiligo months before her wedding due to stress - Metro',
 'Woman spends £1,000 on elf ears implants for her wedding day - Metro',
 "Spiteful sister made bomb hoax to ruin brother's wedding 'because she hated bride' - Mirror.co.uk",
 'Woman marrying her duvet reveals details of lavish wedding ceremony - Mirror Online',
 "Dannii Minogue likes to spend the weekend drinki

In [64]:
df

,Unnamed: 0,Unnamed: 0.1,url,headline,time,scrape_date,site,monthly_visits,country,country_of_pub,...,clean_hl_words_no_women,compound_s,positive_s,negative_s,neutral_s,hl_words,gendered,bias,polarity,year
55100,55100,55100,https://www.dnaindia.com/world/report-british-...,British woman is world's oldest natural mother...,2007-08-20 07:00:00+00:00,15/05/2021,dnaindia.com,7181000,India,India,...,"['british', 'world', 'old', 'natural']",0.3612,0.238,0.000,0.762,"{'mother', 'at', ""world's"", 'british', 'is', '...",1,2,0.3612,2007
55101,55101,55101,https://www.dnaindia.com/india/report-woman-al...,Woman alleges rape by three Swaminarayan sadhu...,2007-08-26 07:00:00+00:00,15/05/2021,dnaindia.com,7181000,India,India,...,"['allege', 'rape', 'swaminarayan', 'sadhu']",-0.6908,0.000,0.439,0.561,"{'rape', 'by', 'alleges', 'swaminarayan', 'thr...",1,1,0.6908,2007
55102,55102,55102,https://www.dnaindia.com/india/report-hindu-wo...,Hindu woman performs last rites of husband - D...,2007-07-22 07:00:00+00:00,15/05/2021,dnaindia.com,7181000,India,India,...,"['hindu', 'perform', 'rite', 'husband']",0.0000,0.000,0.000,1.000,"{'hindu', 'of', 'last', 'rites', 'husband', 'p...",1,1,0.0000,2007
55103,55103,55103,https://www.dnaindia.com/world/report-hasina-b...,Hasina becomes grandmother again - Daily News ...,2007-07-11 07:00:00+00:00,15/05/2021,dnaindia.com,7181000,India,India,...,['hasina'],0.0000,0.000,0.000,1.000,"{'again', 'becomes', 'grandmother', 'hasina'}",1,1,0.0000,2007
55104,55104,55104,https://www.dnaindia.com/entertainment/report-...,Padma Lakshmi..the woman who broke Rushdie's h...,2007-07-05 07:00:00+00:00,15/05/2021,dnaindia.com,7181000,India,India,...,"['padma', 'lakshmi', 'break', 'rushdie', 'heart']",0.3400,0.350,0.233,0.417,"{""rushdie's"", 'lakshmi..the', 'who', 'padma', ...",1,1,0.3400,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277978,277978,277978,https://magazine.outlookindia.com/story/that-g...,That Girl In Yellow Boots - Outlook,2011-09-12 07:00:00+00:00,15/05/2021,outlookindia.com,2385000,India,India,...,"['yellow', 'boot']",0.0000,0.000,0.000,1.000,"{'boots', 'in', 'that', 'girl', 'yellow'}",1,1,0.0000,2011
277981,277981,277981,https://magazine.outlookindia.com/story/got-a-...,"Got A Girl, Named Sue - Outlook",2010-04-26 07:00:00+00:00,15/05/2021,outlookindia.com,2385000,India,India,...,"['get', 'name', 'sue']",0.0000,0.000,0.000,1.000,"{'got', 'a', 'girl,', 'sue', 'named'}",0,0,0.0000,2010
277982,277982,277982,https://magazine.outlookindia.com/story/cook-t...,"Cook, Thief, His Wife And Her Lover - Outlook",2010-04-26 07:00:00+00:00,15/05/2021,outlookindia.com,2385000,India,India,...,"['cook', 'thief', 'wife', 'lover']",0.1027,0.311,0.279,0.410,"{'wife', 'lover', 'and', 'cook,', 'thief,', 'h...",1,2,0.1027,2010
277983,277983,277983,https://magazine.outlookindia.com/story/i-am-a...,"“I Am A Female, And Once Loved A Man” - Outlook",2012-07-30 07:00:00+00:00,15/05/2021,outlookindia.com,2385000,India,India,...,"['love', 'man']",0.5994,0.328,0.000,0.672,"{'a', 'am', 'once', 'and', '“i', 'man”', 'fema...",0,0,0.5994,2012


In [3]:
title = headlines[(headlines['headline'].str.startswith('Women')==True)]
title[title['year']>=2020]['country_of_pub'].unique()

array(['India', 'USA', 'South Africa'], dtype=object)

In [4]:
title = headlines[(headlines['headline'].str.startswith('Woman')==True) | (headlines['headline'].str.startswith('Women')==True) ]
title = title[title['year']>=2020].sort_values(by=['country_of_pub', 'monthly_visits'], ascending = False)

In [29]:
title.to_csv('../data/processed/headlines_starting_woman.csv')

In [20]:
women_headlines = title['headline_no_site'].str.replace('Woman', '').str.replace('Women', '')

In [37]:
women_headlines = women_headlines[~(women_headlines.str.startswith("'"))]

In [54]:
women_headlines = women_headlines[~(women_headlines.str.endswith('...'))]
women_headlines = women_headlines[~(women_headlines.str.contains("'"))]

In [56]:
len(list(women_headlines.drop_duplicates()))

27